В этом ноутбуке кортежи мнений предсказываются с помощью модели gpt-4o или любой другой модели, доступной через сервис **openrouter**.

Здесь используется OpenAI API, также необходим Openrouter API key.

Рекомендуется запускать из Google Colab, а не локально, во избежание региональных ограничений.

## Проверка (OpenAI API) ##

In [2]:
import requests
import ast
import json
import os
import itertools
from tqdm import tqdm
import pandas as pd
from collections import defaultdict
import random
from random import choices
import time

from openai import OpenAI

OPENROUTER_API_KEY = "YOUR_API_KEY"

# model = "thedrummer/unslopnemo-12b:free"
# model = "openai/gpt-4o-2024-11-20"
# model = "x-ai/grok-beta"
model = "deepseek/deepseek-chat"
url = "https://openrouter.ai/api/v1"

SEED = 42
random.seed(SEED)

client = OpenAI(
    api_key=OPENROUTER_API_KEY,
    base_url=url,
)

params = {'temperature': 1.7,
         'top_p': 0.9,
         'max_tokens': 512,
         'seed': SEED}

completion = client.chat.completions.create(
    model=model,
    **params,
    messages=[
        {"role": "system", "content": "Ответь на вопрос."},
        {"role": "user", "content": "Кто тебя создал?"},
    ],
)

print(completion.choices[0].message.content)

Меня создала компания OpenAI, и я основан на технологии GPT-4. Если есть ещё вопросы или что-то, в чём могу помочь, просто дай знать! 😊


## Подготовка данных ##

In [3]:
import requests, zipfile, io

# download utility functions
url = 'https://raw.githubusercontent.com/rossyaykin/RuOpinionNE-2024/refs/heads/main/utils/src.zip'
r = requests.get(url)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall('')

from src.utils import parse_data, load_jsonl, to_jsonl, form_prompt, dict2tuple, extract_tuple, str2list, save_jsonl, df2structure

In [4]:
# download train and test data
train_path = "train.jsonl"
test_path = "test.jsonl"

url = 'https://raw.githubusercontent.com/dialogue-evaluation/RuOpinionNE-2024/refs/heads/master/train.jsonl'
train = load_jsonl(url, train_path)
url = 'https://raw.githubusercontent.com/dialogue-evaluation/RuOpinionNE-2024/refs/heads/master/test.jsonl'
test = load_jsonl(url, test_path)

print(len(train), len(test))

2556 803


In [5]:
# download distances file
dists = requests.get('https://raw.githubusercontent.com/rossyaykin/RuOpinionNE-2024/refs/heads/main/utils/test_distances.txt')
dists = [x.split() for x in dists.text.strip().split('\n')]

print(dists[0][:5])
print(dists[-1][:5])

['307', '1254', '22', '296', '2517']
['1459', '1586', '210', '504', '1438']


In [ ]:
# csv's with preds, generated in Stage1

preds1 = pd.read_csv('PREDS1.csv')
preds2 = pd.read_csv('PREDS2.csv')
preds3 = pd.read_csv('PREDS3.csv')

aug = [preds1, preds2, preds3]

## Определения ##

In [7]:
def aug_prompt(examples, variants, text):
    shots = '\n'.join([f'Текст: {pair[0]}\nОтвет: {pair[1]}' for pair in examples])
    variants = '\n'.join([str(x) for x in variants])
    return f"""Ты эксперт в оценке тональности.
Тебе нужно найти все негативные и позитивные отношения между сущностями в тексте и вывести их в следующем формате:
[источник отношения, объект отношения, выражение в тексте содержащее оценку, оценка (POS/NEG)]
Если источником отношения является автор, то пиши:
['AUTHOR', объект отношения, выражение в тексте содержащее оценку, оценка (POS/NEG)]
Если выраженного источника нет, то пиши:
['NULL', объект отношения, выражение в тексте содержащее оценку, оценка (POS/NEG)]
Допустимо вернуть пустой ответ:
[]
Не нужно давать пояснений к ответу.
Примеры
{shots}
Текст, который нужно проанализировать:
{text}
Ответы экспертов к этому тексту:
{variants}
Ты можешь выбрать из этих ответов или ответить по-своему.
Твой ответ:"""

def aug_prompt_eng(examples, variants, text):
    shots = '\n'.join([f'Текст: {pair[0]}\nОтвет: {pair[1]}' for pair in examples])
    variants = '\n'.join([str(x) for x in variants])
    return f"""You are an expert in sentiment analysis.
You need to identify all positive and negative relations between entities in the text and present them in the following format:
[source of sentiment, target, polar expression, polarity (POS/NEG)]
If the author is the source of sentiment, write:
['AUTHOR', target, polar expression, polarity (POS/NEG)]
If there is no explicit source, write:
['NULL', target, polar expression, polarity (POS/NEG)]
Returning an empty answer is allowed:
[]
Do not provide any explanations in the response.
Examples
{shots}
Text to analyze: {text}
Expert answers to this text:
{variants}
You can choose from these answers or provide your own answer.
Your answer:"""

class Runner():
    def __init__(self, model, client, params, train, test, priorities, aug, n_shots = 5, sleeptime = 2):
        self.model = model
        self.client = client
        self.params = params
        self.train = train
        self.test = test
        self.priorities = priorities
        self.aug = aug
        self.n_shots = n_shots
        self.sleeptime = sleeptime
        self.results = list()

    def run(self):
        if len(self.results) > 0:
            print('Non empty results!')
            return self.results
        
        for i in tqdm(range(len(self.test))):
            time.sleep(self.sleeptime)
            entry = self.test[i]
            # for random choice
            # examples = [dict2tuple(x) for x in choices(self.train, k = self.n_shots)]
            examples = [self.train[int(j)] for j in self.priorities[i][:self.n_shots]]
            examples = [dict2tuple(x) for x in examples]
            variants = [x.loc[i]['pred'] for x in self.aug]
            # English prompt
            # prompt = aug_prompt_eng(examples, variants, entry['text'])
            prompt = aug_prompt(examples, variants, entry['text'])
            
            try:
                completion = client.chat.completions.create(model=self.model,
                                                            **self.params,
                                                            messages=[{"role": "user", "content": prompt}],)
            except JSONDecodeError:
                print(f"JSONDecodeError, iteration {i}")
                return (i, prompt)
            
            result = completion.choices[0].message.content
            try:
                result = ast.literal_eval(result)
            except (SyntaxError, ValueError):
                print(f'bad response, iteration:{i}')
                result = []
            self.results.append((entry['sent_id'],
                            entry['text'],
                            dict2tuple(entry)[1], # gold opinions
                            result)) # pred opinions
        return self.results

def save(dataframe, path, raw = True):
    outdir, outname = '/'.join(path.split('/')[:-1]), path.split('/')[-1]
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    if raw:
        dataframe.to_csv(f'{path}_raw.csv', index = False)
    else:
        dataframe.to_csv(f'{path}.csv', index = False)

## Инференс ##

In [36]:
%%time

# model = "openai/gpt-4o-2024-08-06"
# model = "x-ai/grok-beta"
model = "deepseek/deepseek-chat"

params = {'temperature': 0.1,
          'seed': SEED,
          'top_p': 0.9,
          'max_tokens': 512}
n_shots = 12
runner = Runner(model, client, params, train, test, dists, aug, n_shots)

path = "YOUR_PATH"
result = runner.run()

100%|██████████| 803/803 [1:07:16<00:00,  5.03s/it]

CPU times: total: 5.91 s
Wall time: 1h 7min 16s


In [37]:
path = "YOUR_PATH"
print(path)
print(result[7])

results/Deepseek/deepseek_claug_12shot_0.1temp_eng
(7, 'Назначение командира единого отряда космонавтов, согласно тексту приказа, будет проводиться по согласованию с руководителем Роскосмоса.', [], [])


## Результаты ##

сохраняем csv

In [38]:
output = pd.DataFrame(result, columns = ['sent_id', 'text', 'target', 'pred'])
output

,sent_id,text,target,pred
0,0,"В свою очередь, ""PGNiG намерен требовать реали...",[],[]
1,1,"Известного российского певца Бориса Моисеева, ...",[],"[[NULL, Борис Моисеев, перенес инсульт, NEG], ..."
2,2,"Певец находится в клинике ОАО ""Медицина"", его ...",[],[]
3,3,В России создан единый отряд космонавтов.,[],[]
4,4,Три ранее существовавших отдельных отряда косм...,[],[]
...,...,...,...,...
798,798,"Уилер, выпускник военной Академии в Вест-Пойнт...",[],[]
799,799,"В Америке известен тем, что, будучи председате...",[],"[[NULL, председатель фонда Мемориала войны во ..."
800,800,66-летний Уилер жил вместе с женой в Ньюкасле ...,[],[]
801,801,"Известно, что 28 декабря он должен был приехат...",[],[]


In [39]:
print(path)
save(output, path)

results/Deepseek/deepseek_claug_12shot_0.1temp_eng


In [40]:
output = pd.DataFrame([(x[0], x[1], x[2], str2list(extract_tuple(x[3]))) for x in result],
                      columns = ['sent_id', 'text', 'target', 'pred'])
output.head()

,sent_id,text,target,pred
0,0,"В свою очередь, ""PGNiG намерен требовать реали...",[],[]
1,1,"Известного российского певца Бориса Моисеева, ...",[],"[[NULL, Борис Моисеев, перенес инсульт, NEG], ..."
2,2,"Певец находится в клинике ОАО ""Медицина"", его ...",[],[]
3,3,В России создан единый отряд космонавтов.,[],[]
4,4,Три ранее существовавших отдельных отряда косм...,[],[]


In [41]:
print(path)
save(output, path, raw = False)

results/Deepseek/deepseek_claug_12shot_0.1temp_eng


## csv to jsonl ##

переводим в формат для подачи на RuOpinionNE-2024

In [42]:
final = df2structure(output)
final[13]

{'sent_id': 13,
 'text': 'В Пекине в настоящий момент зарегистрировано 6 кольцевых дорог и 4.76 миллиона транспортных средств, по загруженности китайская столица занимает сомнительное первое место в мире вместе с мексиканском Мехико.',
 'opinions': [{'Source': [['NULL'], ['0:0']],
   'Target': [['Пекин'], ['2:7']],
   'Polar_expression': [['сомнительное первое место в мире'], ['145:177']],
   'Polarity': 'NEG'},
  {'Source': [['NULL'], ['0:0']],
   'Target': [['Мехико'], ['200:206']],
   'Polar_expression': [['сомнительное первое место в мире'], ['145:177']],
   'Polarity': 'NEG'}]}

In [ ]:
# this may cause UnicodeEncodeError when executed locally
save_jsonl(final, path)